### Extração e conversão de dados da API Wonder - Modelo Hierárquico

#### Documentação do API: https://github.com/alipphardt/cdc-wonder-api .

##### Extração e conversão dos dados de mortalidade por câncer de pulmão. Apesar de argumentos para organizar dados por estado americano constem na documentação da API, não é possível usa-los por este meio. Deste modo, o resultados das querys só exibe o total de mortes por ano nos Estados Unidos pela doença.

In [1]:
import csv
import requests
import xmltodict
import pandas as pd

base_data_path = '../data/'

#### Parâmetros de busca:
##### São disponibilizados diversos parâmetros indicados pela documentação. Entretanto, os únicos em que estamos interessados são os parâmetros "by" que dividem os dados por diversos atributos e o parâmetro F_D76.V2 que seleciona a doença que deve ser buscada (neste caso, C34 corresponde ao câncer de pulmão).

In [2]:
# Parâmetros da busca por um dataset, usando o padrão ICD-10 (Internation Classification of Diseases) da API
# Documentação de parâmetros disponíveis em: https://github.com/alipphardt/cdc-wonder-api


# Parâmetros B ("by") - organização dos dados por ano, região, grupo de idade, estado, etc hierarquicamente.
# Mais usados:
# D76.V7  - Gênero
# D76.V8 - Raça
# D76.V1-level1 - Ano
# D76.V1-level2 - Mês
# D76.V2-level3 - Causa da morte (doença especifica)
# D76.V24 - Dia da Semana
# D76.V20 - Com ou sem autópsia
# D76.V21 - Tipo de lugar da morte (hospital, casa da família)

b_parameters = {
    "B_1": "D76.V1-level1", # por ano
    "B_2": "*None*",
    "B_3": "*None*", 
    "B_4": "*None*", 
    "B_5": "*None*"
}

# Parâmetros M ("measure") - quais dados serão medidos como mortes, população, intervalo de confiança de 95%, etc.

m_parameters = {
    "M_1": "D76.M1", # mortes
    "M_2": "D76.M2", # população
    "M_3": "D76.M3", # mortes/100000 habitantes ()
#     "M_31": "D76.M31",
#     "M_32": "D76.M32",
#     "M_41": "D76.M41",
#     "M_42": "D76.M42"
}

# Parâmetros F/I/V - filtros por doença, ano, regiões do censo e parâmetros para organização de dados
# (como ano do censo de urbanização). Alguns destes parâmetros (como a seleção por estado) funciona apenas
# pelo site da API.

f_parameters = {
    "F_D76.V1": ["*All*"], # Ano/mês
    "F_D76.V10": ["*All*"], # Região do censo - não mudar
    "F_D76.V2": ["C34"], # Códigos ICD-10
    "F_D76.V27": ["*All*"], # Regiões HHS - não mudar
    "F_D76.V9": ["*All*"] # Condado estadual - não mudar
}

i_parameters = {
    "I_D76.V1": "*All* (All Dates)",  # Ano/mês
    "I_D76.V10": "*All* (The United States)", # Região do censo - não mudar
    "I_D76.V2": "*All*", # Códigos ICD-10
    "I_D76.V27": "*All* (The United States)", # Regiões HHS - não mudar
    "I_D76.V9": "*All* (The United States)" # Condado estadual - não mudar
}

v_parameters = {
    "V_D76.V1": "",         # Ano/mês
    "V_D76.V10": "",        # Regiões do censo
    "V_D76.V11": "*All*",   # Urbanização de 2006
    "V_D76.V12": "*All*",   # Códigos de lista de causas ICD-10 130 para crianças
    "V_D76.V17": "*All*",   # Origem hispânica
    "V_D76.V19": "*All*",   # Urbanização de 2013
    "V_D76.V2": "",         # Códgios ICD-10
    "V_D76.V20": "*All*",   # Autópsia
    "V_D76.V21": "*All*",   # Lugar da morte
    "V_D76.V22": "*All*",   # Intenção de lesisonar
    "V_D76.V23": "*All*",   # Mecanismo de lesão e causadores
    "V_D76.V24": "*All*",   # Dia da semana
    "V_D76.V25": "*All*",   # Causas induzidas por drogas e álcool
    "V_D76.V27": "",        # Regiões HHS
    "V_D76.V4": "*All*",    # Códigos de lista de causas ICD-10 130
    "V_D76.V5": "*All*",    # Grupos de idade de 10 anos
    "V_D76.V51": "*All*",   # Grupos de idade de 5 anos
    "V_D76.V52": "*All*",   # Grupos de idade de 1 ano
    "V_D76.V6": "00",       # Grupos de idade de crianças
    "V_D76.V7": "*All*",    # Genêro
    "V_D76.V8": "*All*",    # Raça
    "V_D76.V9": ""          # Estado/país
}

# Parâmetros O - outros parâmetros variados, como timeout e título da consulta.

o_parameters = {
    "O_V10_fmode": "freg",    # Ignorar parâmetro v e usar buscador normal
    "O_V1_fmode": "freg",     # Ignorar parâmetro v e usar buscador normal
    "O_V27_fmode": "freg",    # Ignorar parâmetro v e usar buscador normal
    "O_V2_fmode": "freg",     # Ignorar parâmetro v e usar buscador normal
    "O_V9_fmode": "freg",     # Ignorar parâmetro v e usar buscador normal
    "O_aar": "aar_std",       # Razões de idade ajustada
    "O_aar_pop": "0000",      # Seleção de população para razões com idade ajustada
    "O_age": "D76.V5",        # Selecionar grupo de idade (de 10 em 10 anos, crianças, etc)
    "O_javascript": "on",     # Javascript
    "O_location": "D76.V9",   # Variável de localização
    "O_precision": "2",       # Número de casas decimais
    "O_rate_per": "100000",   # Razões calculados por X pessoas
    "O_show_totals": "false",  # Mostrar totais
    "O_timeout": "300",
    "O_title": "Lung cancer deaths by year", # Titulo da resposta
    "O_ucd": "D76.V2",        # Cause de morte
    "O_urban": "D76.V19"      # Categoria de urbanização
}

# Parâmetros VM - parâmetros de ajuste de idade não padronizados

vm_parameters = {
    "VM_D76.M6_D76.V10": "",        # Localização
    "VM_D76.M6_D76.V17": "*All*",   # Origem hispânica
    "VM_D76.M6_D76.V1_S": "*All*",  # Ano
    "VM_D76.M6_D76.V7": "*All*",    # Genêro
    "VM_D76.M6_D76.V8": "*All*"     # Raça
}

# Parâmetros MISC - parâmetros que devem ser incluidos para o funcionamento da API

misc_parameters = {
    "action-Send": "Send",
    "finder-stage-D76.V1": "codeset",
    "finder-stage-D76.V1": "codeset",
    "finder-stage-D76.V2": "codeset",
    "finder-stage-D76.V27": "codeset",
    "finder-stage-D76.V9": "codeset",
    "stage": "request"
}
parameter_list = [b_parameters,m_parameters,f_parameters,i_parameters,v_parameters,o_parameters,vm_parameters,misc_parameters]

##### A classe WonderAPIQuery recebe os parâmetros em formato de lista e então converte estes parâmetros para uma string XML que será usada como argumento da query.
##### O método fetchData() manda um post request para a API, que responde com os dados em formato XML que por sua vez são convertidos para uma tabela.
##### Assim que os dados foram extraídos e convertidos, o método saveResultToCSV() salva os resultados da query.

In [3]:
class WonderAPIQuery():
    def __init__(self,parameters,table_entries):
        self.request_url = "https://wonder.cdc.gov/controller/datarequest/D76"
        self.table_entries = table_entries
        self.setParametersXML(parameters)
    
    def setParametersXML(self,parameters):
        self.xml = '<request-parameters>\n'
        for param in parameters:
            for key in param:
                self.xml += '<parameter>\n'
                self.xml += '<name>' + key + '</name>\n'
                if isinstance(param[key],list):
                    for element in param[key]:
                        self.xml += '<value>' + element + '</value>\n'
                else:
                    self.xml += '<value>' + param[key] + '</value>\n'
                self.xml += '</parameter>\n'
        self.xml += '</request-parameters>\n'
        
    def fetchData(self):
        response = requests.post(self.request_url, data={"request_xml": self.xml, "accept_datause_restrictions": "true"})
        if response.status_code == 200:
            self.data = response.text
            self.convertToList(self.data)
            return self.data
        else:
            print("Algo deu errado. Tente ajustar os parâmetros da query")
        
    def convertToList(self,data):
        datadict = xmltodict.parse(data)
        result = [self.table_entries]
        for listofdicts in datadict['page']['response']['data-table']['r']:
            row = []
            for i,datavalue in enumerate(listofdicts['c']):
                if(i==4):
                    continue
                try:
                    row.append(int(datavalue['@l']))
                except:
                    z = datavalue['@v'].replace(',','')
                    try:
                        row.append(int(z))
                    except:
                        row.append(float(z))
            result.append(row)
        self.data = result
    
    def saveResultToCSV(self,path,filename):
        with open(path+filename+'.csv','w') as newfile:
            wr = csv.writer(newfile, quoting=csv.QUOTE_NONE)
            for row in self.data:
                wr.writerow(row)

In [4]:
db = WonderAPIQuery(parameter_list,["Year","Deaths","Population","CrudeRate"])
data = db.fetchData()
for row in data:
    print(row)
db.saveResultToCSV(base_data_path,'lungcancer-general')

['Year', 'Deaths', 'Population', 'CrudeRate']
[1999, 152063, 279040168, 54.5]
[2000, 155431, 281421906, 55.23]
[2001, 155973, 284968955, 54.73]
[2002, 157630, 287625193, 54.8]
[2003, 157992, 290107933, 54.46]
[2004, 158009, 292805298, 53.96]
[2005, 159220, 295516599, 53.88]
[2006, 158600, 298379912, 53.15]
[2007, 158686, 301231207, 52.68]
[2008, 158592, 304093966, 52.15]
[2009, 158086, 306771529, 51.53]
[2010, 158249, 308745538, 51.26]
[2011, 156957, 311591917, 50.37]
[2012, 157426, 313914040, 50.15]
[2013, 156178, 316128839, 49.4]
[2014, 155529, 318857056, 48.78]
[2015, 153722, 321418820, 47.83]
[2016, 148870, 323127513, 46.07]
[2017, 145849, 325719178, 44.78]
[2018, 142081, 327167434, 43.43]
